# The Cancer Gene Atlas Cancer (TCGA) - Sarcoma (SARC)

Author: Moshe Silverstein <br>
Date: 7-17 <br>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget, seaborn 

Software versions
Python 3.5.5 64bit [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
IPython 6.3.1
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.2
pandas 0.22.0
clustergrammer_widget 1.9.0
seaborn 0.8.1
Thu May 10 16:58:42 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        
        ID = ID+'-'+metaDF.loc[col, 'a_AliquotBarcode'].split('-')[3][0:2]

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF, SampleTypes):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
#         if '01' in str(col.split('-')[3]):
#             sampleType = 'Primary Solid Tumor'
#         elif '11' in str(col.split('-')[3]):
#             sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+SampleTypes[str(col.split('-')[3])],
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

In [9]:
SampleTypes ={
    '01':'Primary Solid Tumor',
    '02':'Recurrent Solid Tumor',
    '03':'Primary Blood Derived Cancer - Peripheral Blood',
    '04':'Recurrent Blood Derived Cancer - Bone Marrow',
    '05':'Additional - New Primary',
    '06':'Metastatic',
    '07':'Additional Metastatic',
    '08':'Human Tumor Original Cells',
    '09':'Primary Blood Derived Cancer - Bone Marrow',
    '10':'Blood Derived Normal',
    '11':'Solid Tissue Normal',
    '12':'Buccal Cell Normal',
    '13':'EBV Immortalized Normal',
    '14':'Bone Marrow Normal',
    '15':'sample type 15',
    '16':'sample type 16',
    '20':'Control Analyte',
    '40':'Recurrent Blood Derived Cancer - Peripheral Blood',
    '50':'Cell Lines',
    '60':'Primary Xenograft Tissue',
    '61':'Cell Line Derived Xenograft Tissue',
    '99':'sample type 99'
             }

# Path to Output Files

In [10]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [11]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [12]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [13]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [14]:
sample_meta = pd.read_csv('Input/metadata_Sarcoma [SARC]', sep='\t', index_col=1)

In [15]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-3B-A9HV,480,SARC,TCGA,47FB776C-7839-4590-8F2D-6D149A0F1782,3B,48.0,NaN,412,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-3B-A9I1,481,SARC,TCGA,A4F68239-5B9F-40D7-9E52-B68EFA170C1C,3B,56.0,NaN,412,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-FX-A76Y,482,SARC,TCGA,3A96E833-996A-4319-83F1-326B96F4EE80,FX,49.0,NaN,363,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-FX-A48G,483,SARC,TCGA,12BFD446-2EE5-4664-A492-C023D134F60F,FX,56.0,NaN,263,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-HB-A3L4,484,SARC,TCGA,520088B0-FFC0-4ECD-B1ED-2339811D7AD2,HB,46.0,NaN,218,Nationwide Children's Hospital,NaN,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
sample_meta.shape

(261, 70)

# Load Data

In [17]:
matrix = pd.read_csv('Input/TCGA_SARC_tpm.tsv', sep='\t', index_col=0)

In [18]:
matrix.head()

,75af221c-4718-4bb0-8c22-45f799251b77,6a47fe73-f55b-42c3-9c36-32e2a99bf13d,1426e74a-b04a-4a1d-a7c4-62db40551bce,71182bcc-0b40-42b0-8831-3a65858fd48b,f99b4a48-244a-4d58-88b2-bedc47daec80,3f7d15d0-8f6f-416e-9635-6eb70613dc6d,3826fde1-abd7-4fa8-92d1-9c9d4c7fb508,6fa6fa69-1f95-428a-9a82-56a7f532318b,a5297436-f594-42da-ac80-f509bb899458,9adf3e86-8644-4346-a00e-d56337a0340d,...,9631c4f7-e460-48d8-87f5-a288739caa39,9a934c6d-e1bd-4222-b8a4-364e14560166,668c6868-a1c4-4de5-a72a-78a8c2bd85b2,f83fe4c1-c509-4431-8fc2-786ac5e97bdc,1a71b42d-3ca9-4eb5-961b-a01132d5405d,dbfbd2b4-a3a5-423e-b775-52923db5cac5,ed95dd3b-0aac-4199-bf9f-f2a24f3170c6,9caa1f89-c955-4b5f-968c-b14885ca30e1,6cec28c3-c65f-4a52-b274-93803f6ac2bd,6cb863e7-dc86-47d8-ab08-306e22ccdb02
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,126.700000,115.52700,144.00300,195.129000,171.135000,106.51400,157.6190,174.5180,145.506000,186.88300,...,210.919000,258.7050,162.5080,277.75200,172.127000,106.26200,134.59500,187.981000,153.34400,80.832200
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,35.807400,29.54040,55.77520,28.824000,41.929300,45.16820,24.6826,33.6484,47.973100,47.33240,...,22.187500,49.8290,23.2024,26.00830,69.608200,60.42360,36.22330,19.660500,20.79500,0.000000
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.820200,8.07769,15.81100,14.833300,22.916100,12.65650,11.5743,17.8612,11.517500,21.53420,...,12.792000,24.2732,5.0326,11.42080,29.560900,15.88230,12.11810,30.377500,11.61070,2.594890
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,15.922800,55.92280,40.69060,30.406800,64.216600,55.13440,56.5004,29.0133,176.026000,16.93260,...,36.884300,18.6983,35.0882,42.45510,83.687100,39.71300,39.46160,23.421300,19.46020,20.013100
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.700315,0.00000,4.64202,0.254368,0.359553,2.38726,61.3006,14.7678,0.328752,1.90762,...,0.885818,23.8527,13.2993,3.14181,0.751312,2.18335,0.05488,0.457239,6.24766,0.068545


In [19]:
matrix.shape

(199169, 265)

# Get Only Protein Coding Genes

In [20]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [21]:
matrix.head()

,75af221c-4718-4bb0-8c22-45f799251b77,6a47fe73-f55b-42c3-9c36-32e2a99bf13d,1426e74a-b04a-4a1d-a7c4-62db40551bce,71182bcc-0b40-42b0-8831-3a65858fd48b,f99b4a48-244a-4d58-88b2-bedc47daec80,3f7d15d0-8f6f-416e-9635-6eb70613dc6d,3826fde1-abd7-4fa8-92d1-9c9d4c7fb508,6fa6fa69-1f95-428a-9a82-56a7f532318b,a5297436-f594-42da-ac80-f509bb899458,9adf3e86-8644-4346-a00e-d56337a0340d,...,9631c4f7-e460-48d8-87f5-a288739caa39,9a934c6d-e1bd-4222-b8a4-364e14560166,668c6868-a1c4-4de5-a72a-78a8c2bd85b2,f83fe4c1-c509-4431-8fc2-786ac5e97bdc,1a71b42d-3ca9-4eb5-961b-a01132d5405d,dbfbd2b4-a3a5-423e-b775-52923db5cac5,ed95dd3b-0aac-4199-bf9f-f2a24f3170c6,9caa1f89-c955-4b5f-968c-b14885ca30e1,6cec28c3-c65f-4a52-b274-93803f6ac2bd,6cb863e7-dc86-47d8-ab08-306e22ccdb02
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,126.700000,115.52700,144.00300,195.129000,171.135000,106.51400,157.6190,174.5180,145.506000,186.88300,...,210.919000,258.7050,162.5080,277.75200,172.127000,106.26200,134.59500,187.981000,153.34400,80.832200
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,35.807400,29.54040,55.77520,28.824000,41.929300,45.16820,24.6826,33.6484,47.973100,47.33240,...,22.187500,49.8290,23.2024,26.00830,69.608200,60.42360,36.22330,19.660500,20.79500,0.000000
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.820200,8.07769,15.81100,14.833300,22.916100,12.65650,11.5743,17.8612,11.517500,21.53420,...,12.792000,24.2732,5.0326,11.42080,29.560900,15.88230,12.11810,30.377500,11.61070,2.594890
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,15.922800,55.92280,40.69060,30.406800,64.216600,55.13440,56.5004,29.0133,176.026000,16.93260,...,36.884300,18.6983,35.0882,42.45510,83.687100,39.71300,39.46160,23.421300,19.46020,20.013100
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.700315,0.00000,4.64202,0.254368,0.359553,2.38726,61.3006,14.7678,0.328752,1.90762,...,0.885818,23.8527,13.2993,3.14181,0.751312,2.18335,0.05488,0.457239,6.24766,0.068545


In [22]:
matrix.shape

(79930, 265)

# Map Sample Meta Data to Sample ID

In [24]:
getSampleID(matrix, meta)

In [25]:
matrix.head()

,TCGA-PC-A5DP-01,TCGA-K1-A42X-02,TCGA-MO-A47R-01,TCGA-PC-A5DM-01,TCGA-KD-A5QS-01,TCGA-DX-A1KU-01,TCGA-DX-A1KY-01,TCGA-DX-A1L0-01,TCGA-DX-A1KZ-01,TCGA-DX-A1L1-01,...,TCGA-DX-A8BJ-01,TCGA-DX-A8BR-01,TCGA-DX-AB3B-01,TCGA-DX-AB35-01,TCGA-DX-AB2Q-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,TCGA-DX-A8BO-01,TCGA-Z4-A8JB-01,TCGA-DX-A7EQ-01
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,126.700000,115.52700,144.00300,195.129000,171.135000,106.51400,157.6190,174.5180,145.506000,186.88300,...,210.919000,258.7050,162.5080,277.75200,172.127000,106.26200,134.59500,187.981000,153.34400,80.832200
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,35.807400,29.54040,55.77520,28.824000,41.929300,45.16820,24.6826,33.6484,47.973100,47.33240,...,22.187500,49.8290,23.2024,26.00830,69.608200,60.42360,36.22330,19.660500,20.79500,0.000000
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.820200,8.07769,15.81100,14.833300,22.916100,12.65650,11.5743,17.8612,11.517500,21.53420,...,12.792000,24.2732,5.0326,11.42080,29.560900,15.88230,12.11810,30.377500,11.61070,2.594890
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,15.922800,55.92280,40.69060,30.406800,64.216600,55.13440,56.5004,29.0133,176.026000,16.93260,...,36.884300,18.6983,35.0882,42.45510,83.687100,39.71300,39.46160,23.421300,19.46020,20.013100
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,0.700315,0.00000,4.64202,0.254368,0.359553,2.38726,61.3006,14.7678,0.328752,1.90762,...,0.885818,23.8527,13.2993,3.14181,0.751312,2.18335,0.05488,0.457239,6.24766,0.068545


# Map Gene Symbol to Gene ID

In [26]:
getGenes(matrix)

In [27]:
matrix.head()

,TCGA-PC-A5DP-01,TCGA-K1-A42X-02,TCGA-MO-A47R-01,TCGA-PC-A5DM-01,TCGA-KD-A5QS-01,TCGA-DX-A1KU-01,TCGA-DX-A1KY-01,TCGA-DX-A1L0-01,TCGA-DX-A1KZ-01,TCGA-DX-A1L1-01,...,TCGA-DX-A8BJ-01,TCGA-DX-A8BR-01,TCGA-DX-AB3B-01,TCGA-DX-AB35-01,TCGA-DX-AB2Q-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,TCGA-DX-A8BO-01,TCGA-Z4-A8JB-01,TCGA-DX-A7EQ-01
ARF5,126.700000,115.52700,144.00300,195.129000,171.135000,106.51400,157.6190,174.5180,145.506000,186.88300,...,210.919000,258.7050,162.5080,277.75200,172.127000,106.26200,134.59500,187.981000,153.34400,80.832200
M6PR,35.807400,29.54040,55.77520,28.824000,41.929300,45.16820,24.6826,33.6484,47.973100,47.33240,...,22.187500,49.8290,23.2024,26.00830,69.608200,60.42360,36.22330,19.660500,20.79500,0.000000
ESRRA,15.820200,8.07769,15.81100,14.833300,22.916100,12.65650,11.5743,17.8612,11.517500,21.53420,...,12.792000,24.2732,5.0326,11.42080,29.560900,15.88230,12.11810,30.377500,11.61070,2.594890
FKBP4,15.922800,55.92280,40.69060,30.406800,64.216600,55.13440,56.5004,29.0133,176.026000,16.93260,...,36.884300,18.6983,35.0882,42.45510,83.687100,39.71300,39.46160,23.421300,19.46020,20.013100
CYP26B1,0.700315,0.00000,4.64202,0.254368,0.359553,2.38726,61.3006,14.7678,0.328752,1.90762,...,0.885818,23.8527,13.2993,3.14181,0.751312,2.18335,0.05488,0.457239,6.24766,0.068545


# Histogram of First Sample

In [28]:
sns.distplot(matrix.iloc[:, 0]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [29]:
sns.distplot(matrix.iloc[0, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Unfiltered Matrix to File

In [30]:
filename = path+'tcga_sarc_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [31]:
normalized_matrix = matrix.copy()

In [32]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining missing values to matrix mean

In [33]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [34]:
normalized_matrix.head()

,TCGA-PC-A5DP-01,TCGA-K1-A42X-02,TCGA-MO-A47R-01,TCGA-PC-A5DM-01,TCGA-KD-A5QS-01,TCGA-DX-A1KU-01,TCGA-DX-A1KY-01,TCGA-DX-A1L0-01,TCGA-DX-A1KZ-01,TCGA-DX-A1L1-01,...,TCGA-DX-A8BJ-01,TCGA-DX-A8BR-01,TCGA-DX-AB3B-01,TCGA-DX-AB35-01,TCGA-DX-AB2Q-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,TCGA-DX-A8BO-01,TCGA-Z4-A8JB-01,TCGA-DX-A7EQ-01
A1BG,0.209103,0.033044,0.209103,0.209103,0.209103,0.209103,0.209103,0.209103,0.209103,0.209103,...,0.209103,0.209103,0.209103,0.209103,0.209103,0.021233,0.209103,0.209103,0.209103,0.057049
A1CF,0.009871,0.010586,0.024663,0.004683,0.004147,0.006917,0.067241,0.020041,0.014060,0.013185,...,0.043601,0.003022,0.013156,0.051470,0.031458,0.011779,0.014376,0.013642,0.046553,0.046553
A2M,938.188000,1072.370000,3191.910000,28.157300,191.414000,1132.510000,237.590000,699.358000,228.759000,294.407000,...,82.997500,442.602000,133.418000,307.682000,107.774000,162.754000,2548.530000,105.981000,188.349000,19.886200
A2ML1,0.166305,0.183088,0.137806,0.080031,0.049954,0.079906,0.135088,0.067919,0.240370,0.039899,...,0.033301,0.099976,0.107710,0.025446,0.074950,0.044579,0.059009,0.017157,0.089471,0.139892
A3GALT2,0.074539,0.074539,0.091177,0.127845,0.074539,0.064874,0.141963,0.036362,0.074539,0.027755,...,0.053468,0.074539,0.082864,0.074539,0.074539,0.074539,0.074539,0.028724,0.074539,0.074539


In [35]:
normalized_matrix.shape

(19103, 265)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [36]:
samples = normalized_matrix.columns.tolist()

In [37]:
normalized_matrix.columns = np.arange(0, len(normalized_matrix.columns))

In [38]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [39]:
normalized_matrix.columns = samples

In [40]:
normalized_matrix.head()

,TCGA-PC-A5DP-01,TCGA-K1-A42X-02,TCGA-MO-A47R-01,TCGA-PC-A5DM-01,TCGA-KD-A5QS-01,TCGA-DX-A1KU-01,TCGA-DX-A1KY-01,TCGA-DX-A1L0-01,TCGA-DX-A1KZ-01,TCGA-DX-A1L1-01,...,TCGA-DX-A8BJ-01,TCGA-DX-A8BR-01,TCGA-DX-AB3B-01,TCGA-DX-AB35-01,TCGA-DX-AB2Q-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,TCGA-DX-A8BO-01,TCGA-Z4-A8JB-01,TCGA-DX-A7EQ-01
A1BG,-0.486569,-1.290915,-0.465941,-0.507331,-0.497126,-0.510908,-0.479921,-0.494519,-0.494465,-0.468568,...,-0.491039,-0.510479,-0.480995,-0.487699,-0.481659,-1.425150,-0.458744,-0.485368,-0.495260,-0.985259
A1CF,-1.812570,-1.785274,-1.394263,-2.157199,-2.199765,-1.991328,-0.972649,-1.512953,-1.666830,-1.668841,...,-1.171908,-2.350599,-1.682248,-1.096506,-1.304289,-1.681036,-1.621451,-1.670853,-1.147670,-1.073558
A2M,3.165361,3.220339,3.717750,1.621900,2.464488,3.222774,2.575548,3.029821,2.544553,2.680020,...,2.107666,2.815175,2.323859,2.680043,2.230495,2.459377,3.627186,2.219500,2.459344,1.557044
A2ML1,-0.586023,-0.547346,-0.647032,-0.924433,-1.118914,-0.928689,-0.669663,-0.982889,-0.433944,-1.187971,...,-1.288943,-0.830942,-0.769099,-1.402439,-0.927245,-1.103024,-1.008185,-1.571292,-0.863938,-0.595715
A3GALT2,-0.934545,-0.937622,-0.826414,-0.721007,-0.945102,-1.019196,-0.648105,-1.254225,-0.942441,-1.345590,...,-1.083303,-0.958455,-0.882988,-0.935675,-0.929635,-0.879771,-0.906720,-1.347482,-0.943236,-0.869124


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [41]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [42]:
normalized_matrix.head()

,TCGA-PC-A5DP-01,TCGA-K1-A42X-02,TCGA-MO-A47R-01,TCGA-PC-A5DM-01,TCGA-KD-A5QS-01,TCGA-DX-A1KU-01,TCGA-DX-A1KY-01,TCGA-DX-A1L0-01,TCGA-DX-A1KZ-01,TCGA-DX-A1L1-01,...,TCGA-DX-A8BJ-01,TCGA-DX-A8BR-01,TCGA-DX-AB3B-01,TCGA-DX-AB35-01,TCGA-DX-AB2Q-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,TCGA-DX-A8BO-01,TCGA-Z4-A8JB-01,TCGA-DX-A7EQ-01
A1BG,-0.543613,-1.301734,-0.578134,-0.454983,-0.449814,-0.515750,-0.612823,-0.595858,-0.524367,-0.479170,...,-0.406121,-0.439295,-0.557486,-0.520398,-0.344951,-1.443945,-0.535427,-0.379331,-0.586230,-0.843995
A1CF,-1.877826,-1.819137,-1.543229,-2.234943,-2.279392,-2.084983,-1.109175,-1.634271,-1.659954,-1.653465,...,-1.136780,-2.332686,-1.746944,-1.137098,-1.205220,-1.667259,-1.564883,-1.625930,-1.296085,-0.939653
A2M,3.201046,3.201046,3.746187,1.666264,2.398747,3.222775,2.560981,3.052617,2.478735,2.624068,...,2.153409,2.785234,2.380067,2.638470,2.212317,2.403365,3.688937,2.226375,2.499025,1.792929
A2ML1,-0.649352,-0.548607,-0.757838,-0.929278,-1.111429,-0.933718,-0.799187,-1.082257,-0.462983,-1.208038,...,-1.250398,-0.762570,-0.830142,-1.480484,-0.801288,-1.113890,-1.019904,-1.522481,-0.991559,-0.364899
A3GALT2,-0.996559,-0.949550,-0.944450,-0.708871,-0.932680,-1.030533,-0.776241,-1.366184,-0.966178,-1.360045,...,-1.037450,-0.886151,-0.942804,-0.984053,-0.803081,-0.881568,-0.930311,-1.297272,-1.073826,-0.702768


# Normalize Matrix (z-score the rows)

In [43]:
uf.zscore(normalized_matrix, 'row')

In [44]:
normalized_matrix.head()

,TCGA-PC-A5DP-01,TCGA-K1-A42X-02,TCGA-MO-A47R-01,TCGA-PC-A5DM-01,TCGA-KD-A5QS-01,TCGA-DX-A1KU-01,TCGA-DX-A1KY-01,TCGA-DX-A1L0-01,TCGA-DX-A1KZ-01,TCGA-DX-A1L1-01,...,TCGA-DX-A8BJ-01,TCGA-DX-A8BR-01,TCGA-DX-AB3B-01,TCGA-DX-AB35-01,TCGA-DX-AB2Q-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,TCGA-DX-A8BO-01,TCGA-Z4-A8JB-01,TCGA-DX-A7EQ-01
A1BG,-0.370083,-6.881524,-0.666581,0.391158,0.435549,-0.130766,-0.964522,-0.818810,-0.204780,0.183415,...,0.810827,0.525896,-0.489235,-0.170688,1.336212,-8.102968,-0.299767,1.040924,-0.736112,-2.950038
A1CF,-0.547407,-0.409641,0.238028,-1.385706,-1.490046,-1.033688,1.256931,0.024316,-0.035971,-0.020739,...,1.192132,-1.615150,-0.240173,1.191384,1.031476,-0.053120,0.187199,0.043895,0.818176,1.654868
A2M,0.952458,0.952458,1.920576,-1.773166,-0.472347,0.991047,-0.184235,0.688863,-0.330296,-0.072200,...,-0.908044,0.214016,-0.505520,-0.046622,-0.803429,-0.464147,1.818905,-0.778463,-0.294264,-1.548220
A2ML1,0.790572,1.087406,0.470925,-0.034206,-0.570898,-0.047288,0.349094,-0.484946,1.339690,-0.855549,...,-0.980358,0.456983,0.257888,-1.658285,0.342904,-0.578148,-0.301228,-1.782026,-0.217710,1.628684
A3GALT2,-0.254326,0.003072,0.030994,1.320891,0.095442,-0.440348,0.952015,-2.278180,-0.087973,-2.244568,...,-0.478221,0.350207,0.040008,-0.185847,0.805054,0.375299,0.108411,-1.900855,-0.677396,1.354308


# Merge Like Column (by taking the mean)

In [45]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [46]:
normalized_matrix.shape

(19103, 265)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [47]:
uf.mapgenesymbols(normalized_matrix)

In [48]:
normalized_matrix.shape

(18675, 265)

# Merge Duplicate Genes By Rows

In [49]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [50]:
normalized_matrix.shape

(18584, 265)

# Histogram of First Sample

In [51]:
sns.distplot(normalized_matrix.iloc[:, 10]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Histogram of First Gene

In [52]:
sns.distplot(normalized_matrix.iloc[10, :]);

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


# Save Filtered Matrix

In [53]:
filename = path+'tcga_sarc_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [54]:
gene_list = uf.createGeneList(normalized_matrix)

In [55]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [56]:
gene_list.shape

(18584, 2)

# Save Gene List

In [57]:
filename = path+'tcga_sarc_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [58]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    if ('-').join(index.split('-')[0:3]) in sample_meta.index:
        metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
        metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
        metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
        metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
        metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
        metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
        metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
        metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    else:
        metaData.loc[index, 'Age'] = np.nan
        metaData.loc[index, 'Gender'] = np.nan
        metaData.loc[index, 'Pathologic M'] = np.nan
        metaData.loc[index, 'Pathologic N'] = np.nan
        metaData.loc[index, 'Pathologic T'] = np.nan
        metaData.loc[index, 'Pathologic Stage'] = np.nan
        metaData.loc[index, 'Histological Type'] = np.nan
        metaData.loc[index, 'Sample Type'] = SampleTypes[str(index.split('-')[3])]
        metaData.loc[index, 'Tumor Tissue Site'] = np.nan
        metaData.loc[index, 'Neoplasm Histologic Grade'] = np.nan

In [59]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [60]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-3B-A9HI-01,68,MALE,NaN,NaN,NaN,NaN,Dedifferentiated liposarcoma,Primary Solid Tumor,Upper Extremity - Upper arm/elbow,NaN
TCGA-3B-A9HJ-01,68,MALE,NaN,NaN,NaN,NaN,Dedifferentiated liposarcoma,Primary Solid Tumor,Lower Extremity - Groin,NaN
TCGA-3B-A9HL-01,67,MALE,NaN,NaN,NaN,NaN,Dedifferentiated liposarcoma,Primary Solid Tumor,Retroperitoneum/Upper abdominal - Retroperitoneum,NaN
TCGA-3B-A9HO-01,75,MALE,NaN,NaN,NaN,NaN,Dedifferentiated liposarcoma,Primary Solid Tumor,Retroperitoneum/Upper abdominal - Intraabdominal,NaN
TCGA-3B-A9HP-01,57,FEMALE,NaN,NaN,NaN,NaN,Leiomyosarcoma (LMS),Primary Solid Tumor,Retroperitoneum/Upper abdominal - Retroperitoneum,NaN


In [61]:
attribute_list.shape

(265, 10)

# Save Attribute List

In [62]:
filename = path+'tcga_sarc_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [63]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [64]:
standard_matrix.head()

,TCGA-3B-A9HI-01,TCGA-3B-A9HJ-01,TCGA-3B-A9HL-01,TCGA-3B-A9HO-01,TCGA-3B-A9HP-01,TCGA-3B-A9HQ-01,TCGA-3B-A9HR-01,TCGA-3B-A9HS-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,...,TCGA-X6-A8C4-01,TCGA-X6-A8C5-01,TCGA-X6-A8C6-01,TCGA-X6-A8C7-01,TCGA-X9-A971-01,TCGA-X9-A973-01,TCGA-Z4-A8JB-01,TCGA-Z4-A9VC-01,TCGA-Z4-AAPF-01,TCGA-Z4-AAPG-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,-0.124442,0.728537,-0.637412,0.834281,-0.252712,-0.169710,0.441607,0.109526,-0.886415,-0.275335,...,0.932530,0.388756,-0.388501,-0.463907,-0.735521,0.290639,-0.479004,0.350992,-0.629868,0.932530
A1CF,0.041081,-0.705795,-0.471929,-0.864260,-0.766181,-0.645446,0.267518,-0.147559,-0.056980,0.199580,...,0.131676,0.969867,0.516642,0.531738,0.478898,-0.992409,0.637446,0.448678,0.388271,-0.901985
A2M,0.267518,-0.087186,0.199580,-0.479490,0.697866,0.584569,0.456224,-0.132472,-0.404038,0.962323,...,-0.984892,0.780947,0.282608,-0.879371,0.463778,0.109035,-0.260747,0.259953,-0.336192,0.252411
A2ML1,-0.144265,0.067012,0.942901,-0.446044,0.497270,-0.423377,0.112323,-0.249914,-0.476203,-0.227227,...,-0.136746,0.202859,0.550128,0.489759,0.512371,0.897533,-0.189561,0.376460,-0.664797,-0.363061
A3GALT2,-0.599244,0.593083,-0.531358,0.623276,0.796992,0.276017,-0.486101,0.291083,0.389198,0.140123,...,0.993441,-0.878438,-0.916144,0.955674,0.857409,0.940541,-0.357873,0.872521,-0.961378,-0.365421


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [65]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [66]:
filename = path+'tcga_sarc_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [67]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [68]:
tertiary_matrix.head()

,TCGA-3B-A9HI-01,TCGA-3B-A9HJ-01,TCGA-3B-A9HL-01,TCGA-3B-A9HO-01,TCGA-3B-A9HP-01,TCGA-3B-A9HQ-01,TCGA-3B-A9HR-01,TCGA-3B-A9HS-01,TCGA-3B-A9HT-01,TCGA-3B-A9HU-01,...,TCGA-X6-A8C4-01,TCGA-X6-A8C5-01,TCGA-X6-A8C6-01,TCGA-X6-A8C7-01,TCGA-X9-A971-01,TCGA-X9-A973-01,TCGA-Z4-A8JB-01,TCGA-Z4-A9VC-01,TCGA-Z4-AAPF-01,TCGA-Z4-AAPG-01
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,-1,0,0,0,0
A2M,0,0,0,0,0,0,0,0,0,1,...,-1,0,0,0,0,0,0,0,0,0
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,-1,0


# Save Teriary Matrix

In [69]:
filename = path+'tcga_sarc_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [70]:
name = 'tcga_sarc_gene_up_set'

In [71]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [72]:
name = 'tcga_sarc_gene_down_set'

In [73]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [74]:
name = 'tcga_sarc_attribute_up_set'

In [75]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [76]:
name = 'tcga_sarc_attribute_down_set'

In [77]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity Matrix

In [78]:
attribute_normalized_matrix = normalized_matrix.copy()

In [79]:
getSampleData(attribute_normalized_matrix, sample_meta, SampleTypes)

In [80]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [81]:
attribute_similarity_matix.head()

,"(Donor:TCGA-3B-A9HI, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Dedifferentiated liposarcoma, Tissue:Upper Extremity - Upper arm/elbow, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HJ, Age:68.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Dedifferentiated liposarcoma, Tissue:Lower Extremity - Groin, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HL, Age:67.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Dedifferentiated liposarcoma, Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HO, Age:75.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Dedifferentiated liposarcoma, Tissue:Retroperitoneum/Upper abdominal - Intraabdominal, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HP, Age:57.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HQ, Age:66.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HR, Age:38.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Lower Extremity - Thigh/knee, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HS, Age:60.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HT, Age:53.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Head and Neck - Other (please specify, Tumor:nan, Gradenan)","(Donor:TCGA-3B-A9HU, Age:53.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)",...,"(Donor:TCGA-X6-A8C4, Age:70.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Undifferentiated Pleomorphic Sarcoma (UPS), Tissue:Lower Extremity - Lower leg/calf, Tumor:nan, Gradenan)","(Donor:TCGA-X6-A8C5, Age:63.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Retroperitoneum/Upper abdominal - Small Intestines, Tumor:nan, Gradenan)","(Donor:TCGA-X6-A8C6, Age:55.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Myxofibrosarcoma, Tissue:Lower Extremity - Thigh/knee, Tumor:nan, Gradenan)","(Donor:TCGA-X6-A8C7, Age:24.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Synovial Sarcoma - Biphasic, Tissue:Lower Extremity - Foot/ankle, Tumor:nan, Gradenan)","(Donor:TCGA-X9-A971, Age:52.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Leiomyosarcoma (LMS), Tissue:Upper Extremity - Shoulder/axilla, Tumor:nan, Gradenan)","(Donor:TCGA-X9-A973, Age:55.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Undifferentiated Pleomorphic Sarcoma (UPS), Tissue:Lower abdominal/Pelvic - Pelvic, Tumor:nan, Gradenan)","(Donor:TCGA-Z4-A8JB, Age:24.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Desmoid Tumor, Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)","(Donor:TCGA-Z4-A9VC, Age:37.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Dedifferentiated liposarcoma, Tissue:Lower Extremity - Thigh/knee, Tumor:nan, Gradenan)","(Donor:TCGA-Z4-AAPF, Age:35.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Sarcoma; synovial; poorly differentiated, Tissue:Upper Extremity - Upper arm/elbow, Tumor:nan, Gradenan)","(Donor:TCGA-Z4-AAPG, Age:64.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Undifferentiated Pleomorphic Sarcoma (UPS), Tissue:Retroperitoneum/Upper abdominal - Retroperitoneum, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-3B-A9HI, Age:68.0, Ge

# Save Attribute Similarity Matrix

In [82]:
filename = path+'tcga_sarc_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# HeatMap (clustergrammer) of Similarity Matrix

In [83]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

clustergrammer_widget(network='{"row_nodes": [{"ini": 265, "cat-4": "Histology:Dedifferentiated liposarcoma", …

# Create Gene Similarity Matrix

In [84]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [85]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,0.128550,-0.046809,0.056295,0.028365,0.028165,0.083672,-0.017033,0.027203,0.126533,...,-0.028843,0.041148,0.106997,0.085288,0.041489,-0.046820,0.091385,0.011565,0.055232,0.094768
A1CF,0.128550,1.000000,-0.048628,-0.053913,0.000592,-0.037993,0.033767,-0.021843,-0.117934,-0.016037,...,-0.067755,-0.060419,0.043767,0.035573,0.003015,0.058776,0.039587,-0.069503,0.138549,-0.013300
A2M,-0.046809,-0.048628,1.000000,0.143774,-0.103740,0.390152,-0.021077,-0.325426,0.132377,0.038419,...,-0.341051,-0.104668,0.278881,0.145973,0.160988,-0.280964,0.230547,0.355922,0.065125,0.133323
A2ML1,0.056295,-0.053913,0.143774,1.000000,0.054758,-0.054771,-0.050613,0.015436,0.047707,0.075448,...,-0.153825,-0.057088,0.104047,0.134428,0.130930,-0.008179,-0.005979,0.129884,-0.004521,-0.046263
A3GALT2,0.028365,0.000592,-0.103740,0.054758,1.000000,0.012189,0.026580,0.064246,-0.022864,0.130081,...,-0.009019,0.005679,-0.022551,0.010265,-0.039743,0.028442,-0.082733,0.109939,0.080084,-0.005496


# Save Gene Similarity Matrix

In [86]:
filename = path+'tcga_sarc_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [87]:
name = 'tcga_sarc_gene_attribute_edge_list'

In [88]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  265 Out of 265   

 The number of statisticaly relevent gene-attribute associations is: 245886
